In [1]:
%%capture
!pip install folium matplotlib mapclassify

In [2]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

pois = ['police',
        'fire_station',
        'post_office',
        'library',
        'university',
        'theatre',
        'cinema',
        'playground',
        'art',
        'museum',
        'monument',
        'drinking_water',
        'bench',
        'camera_surveillance']

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


# Import OSM POIs data

In [5]:
osm = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed', 'OSM','gis_osm_pois_free_1.shp'))
osm = osm[osm['fclass'].isin(pois)]
osm = osm[['fclass','geometry']]
osm

,fclass,geometry
6,bench,POINT (9.24015 45.41656)
7,bench,POINT (9.24029 45.41661)
8,drinking_water,POINT (9.22060 45.43419)
15,bench,POINT (9.22385 45.43145)
17,drinking_water,POINT (9.22395 45.43459)
...,...,...
25163,bench,POINT (9.20123 45.52833)
25165,bench,POINT (9.20124 45.52837)
25166,bench,POINT (9.20125 45.52841)
25167,bench,POINT (9.20126 45.52844)


#Sjoin

In [6]:
pois = nil.sjoin(osm, how='left').drop('index_right',axis=1)
pois

,ID_NIL,NIL,area_sqkm,Residenti,geometry,fclass
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",bench
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",bench
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",bench
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",bench
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",museum
...,...,...,...,...,...,...
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",drinking_water
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",drinking_water
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",drinking_water
87,88,parco bosco in citta',7.834006,690,"POLYGON ((9.07309 45.50604, 9.07304 45.50606, ...",drinking_water


# Groupby

In [7]:
for poi in pois['fclass'].unique():
  if poi is not None:
    pois[poi] = pois.groupby('ID_NIL')['fclass'].transform(lambda x: x[x == poi].count())
pois = pois.drop(['geometry','fclass'], axis=1)
pois = pois.drop_duplicates()
pois

,ID_NIL,NIL,area_sqkm,Residenti,bench,museum,drinking_water,camera_surveillance,theatre,post_office,library,university,monument,cinema,police,fire_station,playground,NaN
0,1,duomo,2.341616,16608,232,24,15,41,7,7,9,2,8,4,2,1,0,0
1,2,brera,1.637333,18294,69,6,10,26,2,6,2,2,2,1,3,0,4,0
2,3,giardini p.ta venezia,0.249637,43,14,1,7,1,0,0,1,0,0,0,0,0,2,0
3,4,guastalla,1.547962,15528,46,2,7,11,2,4,3,0,2,1,0,0,2,0
4,5,porta vigentina - porta lodovica,1.135196,13548,82,0,9,5,0,1,2,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,parco nord,1.532331,98,36,0,4,0,0,0,0,0,1,0,0,0,0,0
84,85,parco delle abbazie,13.733841,364,2,0,1,0,0,0,0,0,0,0,0,0,0,0
85,86,parco dei navigli,3.623149,354,0,0,0,0,0,0,0,0,0,0,1,0,0,0
86,87,assiano,5.840942,211,0,0,2,1,0,0,0,0,0,0,0,0,0,0


#Indexes